# **House Price Prediction and Impacting Factors in England**

*This paper* 
>每个section添加下面的链接


| [1. Introduction](#1.-Introduction) | [2. Literature Review](#2.-Literature-Review) | [3. Research Question](#3.-Research-Question) | [4. Data](#4.-Data) | [5. Methodology](#5.-Methodology) | [6. Results and Discussion](#6.-Results-and-Discussion) | [7. Conclusion](#7.-Conclusion) | [Bibliography](#Bibliography) |

---

## 1. Introduction


house price prediction is regression problem
supervised learning

In [1]:
# Import time for measuring the execution time of the program
import time

start_time = time.time()

In [2]:
# Import the packages for data manipulation and processing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

pd.set_option("display.max_rows", 300)  # specifies number of rows to show
pd.options.display.float_format = (
    "{:40,.4f}".format
)  # specifies default number format to 4 decimal places
plt.style.use("ggplot")  # specifies that graphs should use ggplot styling
%matplotlib inline

## 2. Literature Review

## 3. Research Question

## 4. Data

The data is obtained from HM Land Registry Open Data, which contains house price paid data in England and Wales in 2021

### 4.1 Data Cleaning

add column headers, 
import dataset from
from the data info, it can be seen that the transaction unique identifier is object, which can be convert to integer transaction ID for further analysis

In [3]:
# Import the dataset
df = pd.read_csv("Data/Price_Paid_Data_2021.csv")
df

,Transaction_unique_identifier,Price,Date_of_Transfer,Postcode,Property_Type,Building_Age,Tenure,PAON,SAON,Street,Locality,Town/City,District,County,PPD_Category_Type
0,{C6209F5F-3182-295E-E053-6C04A8C0DDCC},270000,2021/3/11 0:00,CM18 7JN,T,N,F,23,NaN,CORNER MEADOW,NaN,HARLOW,HARLOW,ESSEX,A
1,{C6209F5F-3184-295E-E053-6C04A8C0DDCC},246000,2021/6/4 0:00,SS8 8JW,D,N,F,22,NaN,LIMETREE ROAD,NaN,CANVEY ISLAND,CASTLE POINT,ESSEX,A
2,{C6209F5F-3185-295E-E053-6C04A8C0DDCC},250000,2021/4/9 0:00,CM2 8QL,T,N,F,25,NaN,THE STREET,GALLEYWOOD,CHELMSFORD,CHELMSFORD,ESSEX,A
3,{C6209F5F-3186-295E-E053-6C04A8C0DDCC},660000,2021/5/26 0:00,CO6 1QU,D,N,F,3,NaN,GARDENERS ROW,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A
4,{C6209F5F-3187-295E-E053-6C04A8C0DDCC},567500,2021/4/9 0:00,SS6 9NS,D,N,F,6,NaN,SAXON CLOSE,NaN,RAYLEIGH,ROCHFORD,ESSEX,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944722,{CD5A9DCB-FAD5-310A-E053-6C04A8C00A1F},325000,2021/5/28 0:00,ME10 4AG,T,N,F,19,NaN,HONESTY CLOSE,NaN,SITTINGBOURNE,SWALE,KENT,A
944723,{CD5A9DCB-FAD8-310A-E053-6C04A8C00A1F},355000,2021/5/28 0:00,ME12 3TD,D,N,F,31,NaN,CHARLOCK DRIVE,MINSTER ON SEA,SHEERNESS,SWALE,KENT,A
944724,{CD5A9DCB-FAD9-310A-E053-6C04A8C00A1F},540000,2021/6/25 0:00,TN9 2NJ,D,N,F,93,NaN,JUDD ROAD,NaN,TONBRIDGE,TONBRIDGE AND MALLING,KENT,A
944725,{CD5A9DCB-FADA-310A-E053-6C04A8C00A1F},175000,2021/7/14 0:00,CT17 9PU,D,N,F,7,NaN,QUEENS AVENUE,NaN,DOVER,DOVER,KENT,A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944727 entries, 0 to 944726
Data columns (total 15 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Transaction_unique_identifier  944727 non-null  object
 1   Price                          944727 non-null  int64 
 2   Date_of_Transfer               944727 non-null  object
 3   Postcode                       941896 non-null  object
 4   Property_Type                  944727 non-null  object
 5   Building_Age                   944727 non-null  object
 6   Tenure                         944727 non-null  object
 7   PAON                           944727 non-null  object
 8   SAON                           103127 non-null  object
 9   Street                         926258 non-null  object
 10  Locality                       360019 non-null  object
 11  Town/City                      944727 non-null  object
 12  District                       944727 non-nu

There are some house postcodes missing from the summary table, which will be excluded. And since only England data will be investigated, Wales data will be excluded.

In [5]:
# Drop rows with NA postcode
df = df.dropna(subset = ['Postcode']) 

# Drop rows with Wales Postcode
df = df[df.Postcode.str.startswith(('CF', 'LD', 'LL', 'NP', 'SA', 'SY')) == False]

df

,Transaction_unique_identifier,Price,Date_of_Transfer,Postcode,Property_Type,Building_Age,Tenure,PAON,SAON,Street,Locality,Town/City,District,County,PPD_Category_Type
0,{C6209F5F-3182-295E-E053-6C04A8C0DDCC},270000,2021/3/11 0:00,CM18 7JN,T,N,F,23,NaN,CORNER MEADOW,NaN,HARLOW,HARLOW,ESSEX,A
1,{C6209F5F-3184-295E-E053-6C04A8C0DDCC},246000,2021/6/4 0:00,SS8 8JW,D,N,F,22,NaN,LIMETREE ROAD,NaN,CANVEY ISLAND,CASTLE POINT,ESSEX,A
2,{C6209F5F-3185-295E-E053-6C04A8C0DDCC},250000,2021/4/9 0:00,CM2 8QL,T,N,F,25,NaN,THE STREET,GALLEYWOOD,CHELMSFORD,CHELMSFORD,ESSEX,A
3,{C6209F5F-3186-295E-E053-6C04A8C0DDCC},660000,2021/5/26 0:00,CO6 1QU,D,N,F,3,NaN,GARDENERS ROW,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A
4,{C6209F5F-3187-295E-E053-6C04A8C0DDCC},567500,2021/4/9 0:00,SS6 9NS,D,N,F,6,NaN,SAXON CLOSE,NaN,RAYLEIGH,ROCHFORD,ESSEX,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944722,{CD5A9DCB-FAD5-310A-E053-6C04A8C00A1F},325000,2021/5/28 0:00,ME10 4AG,T,N,F,19,NaN,HONESTY CLOSE,NaN,SITTINGBOURNE,SWALE,KENT,A
944723,{CD5A9DCB-FAD8-310A-E053-6C04A8C00A1F},355000,2021/5/28 0:00,ME12 3TD,D,N,F,31,NaN,CHARLOCK DRIVE,MINSTER ON SEA,SHEERNESS,SWALE,KENT,A
944724,{CD5A9DCB-FAD9-310A-E053-6C04A8C00A1F},540000,2021/6/25 0:00,TN9 2NJ,D,N,F,93,NaN,JUDD ROAD,NaN,TONBRIDGE,TONBRIDGE AND MALLING,KENT,A
944725,{CD5A9DCB-FADA-310A-E053-6C04A8C00A1F},175000,2021/7/14 0:00,CT17 9PU,D,N,F,7,NaN,QUEENS AVENUE,NaN,DOVER,DOVER,KENT,A


From all the available features, the most relevant factors are identified for building the prediction model, and the rest are excluded as they are missing, duplicate or redundant. The variables listed below are those that contribute to house price forecasting. It should be noticed that postcode is decided to represent property location rather than the other position information, which is because postcode is succinct and unique, and it will be convert to longitude and latittude in the later process.
- **Price**: stated sale price of the properties (*dependent variable*)
- **Date of Transfer**: date when the sale is finished, Year/Month/Day_Time (*independent variable*)
- **Postcode**: postcode of th properties (*independent variable*)
- **Property type**: D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other  (*independent variable*)
- **Building age**: the age of the property, Y = a newly built property, N = an established residential building (*independent variable*)
- **Tenure**: tenure of the properties, F = Freehold, L= Leasehold (*independent variable*)


In [6]:
df = df.drop([
        "Transaction_unique_identifier",
        "PAON",
        "SAON",
        "Street",
        "Locality",
        "Town/City",
        "District",
        "County",
        "PPD_Category_Type",],axis=1,)

df.head()

,Price,Date_of_Transfer,Postcode,Property_Type,Building_Age,Tenure
0,270000,2021/3/11 0:00,CM18 7JN,T,N,F
1,246000,2021/6/4 0:00,SS8 8JW,D,N,F
2,250000,2021/4/9 0:00,CM2 8QL,T,N,F
3,660000,2021/5/26 0:00,CO6 1QU,D,N,F
4,567500,2021/4/9 0:00,SS6 9NS,D,N,F


It could be seen that the data is still extremely large, using the entire dataset would be computationally challenging and significantly affect the program running and training time. To improve the time and training efficiency, only 50 thousand sets of data is selected. The data with na values is excluded, then the datasets will be randomly selected to avoid data bias and inequity.

In [7]:
df = df.dropna() 

In [8]:
np.random.seed(10)

df_subset = df.sample(n=50000)

df_subset

,Price,Date_of_Transfer,Postcode,Property_Type,Building_Age,Tenure
766120,72500,2021/4/28 0:00,TS12 2AL,T,N,F
513417,110000,2021/3/23 0:00,ST4 7EW,S,N,F
643478,261000,2021/3/4 0:00,PO1 3FT,F,N,L
834396,135000,2021/5/5 0:00,CO4 9YD,F,N,F
559621,370000,2021/8/6 0:00,CV5 9DW,S,N,F
...,...,...,...,...,...,...
239009,215000,2021/6/25 0:00,WA3 2XE,S,N,F
824155,90000,2021/4/26 0:00,S41 0BU,S,N,F
886556,328000,2021/9/23 0:00,M27 0FA,S,N,F
814395,360000,2021/6/24 0:00,DH8 9EE,O,N,F


### 4.2 Data Investigating and Pre-processing

In order to apply the data for establishing prediction model, pre-processing is required.

Firstly, 

The date of transaction is now represented as the number of months since January 1995 (the earliest month of transaction in our dataset), to simplify the representation of time by months only, instead of month and year.




In [9]:
# Transfer the date data to month data since 2021 Janurary
df_subset['Month'] = pd.DatetimeIndex(df_subset['Date_of_Transfer']).month 

# Drop the date of transfer column
df_subset = df_subset.drop(['Date_of_Transfer'], axis=1)

df_subset.head()

,Price,Postcode,Property_Type,Building_Age,Tenure,Month
766120,72500,TS12 2AL,T,N,F,4
513417,110000,ST4 7EW,S,N,F,3
643478,261000,PO1 3FT,F,N,L,3
834396,135000,CO4 9YD,F,N,F,5
559621,370000,CV5 9DW,S,N,F,8


Secondly, the postcode is replaced by latitudes and longitudes to indicate the

 import geocoder
 To capture geographical variation in housing prices more accurately, we decide to represent the address of the property in terms of latitudes and longitudes rather than postcodes. This is preferable to postcodes as neigh- bouring regions do not necessarily share similar postcodes (e.g. SW7 and W8). This process is automated through a Python script which looks up the postcode in a pre-compiled CSV file of London postcodes1, and returns the corresponding latitude and longitude.

In [10]:
# read UK coordinates file into pandas DataFrame
GBcoords1 = pd.read_csv("Data/GBcoords1.csv")
GBcoords2 = pd.read_csv("Data/GBcoords2.csv")


In [11]:
# Concat the two coordinate file
frames = [GBcoords1, GBcoords2]

GBcoords = pd.concat(frames)

In [12]:
# Merge the latitude and longitude data into the datasets
df_subset  = df_subset.merge(GBcoords, on='Postcode', how='left')
df_subset

,Price,Postcode,Property_Type,Building_Age,Tenure,Month,Latitude,Longitude
0,72500,TS12 2AL,T,N,F,4,54.5575,-0.9597
1,110000,ST4 7EW,S,N,F,3,53.0079,-2.1951
2,261000,PO1 3FT,F,N,L,3,50.8005,-1.1029
3,135000,CO4 9YD,F,N,F,5,51.9100,0.9163
4,370000,CV5 9DW,S,N,F,8,52.4332,-1.5664
...,...,...,...,...,...,...,...,...
49995,215000,WA3 2XE,S,N,F,6,53.4695,-2.5806
49996,90000,S41 0BU,S,N,F,4,53.2294,-1.3998
49997,328000,M27 0FA,S,N,F,9,53.5005,-2.3537
49998,360000,DH8 9EE,O,N,F,6,54.8338,-1.8117


In [13]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Price          50000 non-null  int64  
 1   Postcode       50000 non-null  object 
 2   Property_Type  50000 non-null  object 
 3   Building_Age   50000 non-null  object 
 4   Tenure         50000 non-null  object 
 5   Month          50000 non-null  int64  
 6   Latitude       49977 non-null  float64
 7   Longitude      49977 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 3.4+ MB


In [15]:
# Drop the rows without latitude and longitude values (latitude and longitude are consistent)
df_subset = df_subset.dropna(subset = ['Latitude']) 

# Drop the Postcode column
df_subset = df_subset.drop(['Postcode'], axis=1)

df_subset.head()

KeyError: "['Postcode'] not found in axis"

Finally, 

Binary categorical variables (build & tenure) are represented using one binary digit (i.e. (Build) 0 = Old, 1 = New / (Tenure) 0 = Leasehold, 1 = Freehold), while property type is represented using three binary digits through the use of dummy coding [43] (i.e. 000 = Detached, 100 = Semi- detached, 010 = Terrace, 001 = Flat).

I want Old =0 and New =1 So we drop column "N" here

In [16]:
# Data of Building age is converted into Binary Data
df_one = pd.get_dummies(df_subset['Building_Age'])
 
# Binary Data is Concatenated into Dataframe
df_two = pd.concat((df_subset, df_one), axis=1)
 
# Column "Building_Age" is dropped
df_two = df_two.drop(['Building_Age'], axis=1)
 
# I want Old =0 and New =1 So we drop column "N" here
df_two = df_two.drop(["N"], axis=1)
 
# Rename the Column
df_numeric1 = df_two.rename(columns={"Y": "Building_Age"})
 
# Print the Result
df_numeric1

,Price,Property_Type,Tenure,Month,Latitude,Longitude,Building_Age
0,72500,T,F,4,54.5575,-0.9597,0
1,110000,S,F,3,53.0079,-2.1951,0
2,261000,F,L,3,50.8005,-1.1029,0
3,135000,F,F,5,51.9100,0.9163,0
4,370000,S,F,8,52.4332,-1.5664,0
...,...,...,...,...,...,...,...
49995,215000,S,F,6,53.4695,-2.5806,0
49996,90000,S,F,4,53.2294,-1.3998,0
49997,328000,S,F,9,53.5005,-2.3537,0
49998,360000,O,F,6,54.8338,-1.8117,0


Data of Tenure is converted into Binary Data by the same way
Leasehold =0 and Freehold =1 So drop column "L" here

In [17]:
# Data of Tenure is converted into Binary Data
df_three = pd.get_dummies(df_numeric1['Tenure'])
 
# Binary Data is Concatenated into Dataframe
df_four = pd.concat((df_numeric1, df_three), axis=1)
 
# Column "Tenure" is dropped
df_four = df_four.drop(['Tenure'], axis=1)
 
# Leasehold =0 and Freehold =1 So drop column "L" here
df_four = df_four.drop(["L"], axis=1)
 
# Rename the Column
df_numeric2 = df_four.rename(columns={"F": "Tenure"})
 
# Print the Result
df_numeric2

,Price,Property_Type,Month,Latitude,Longitude,Building_Age,Tenure
0,72500,T,4,54.5575,-0.9597,0,1
1,110000,S,3,53.0079,-2.1951,0,1
2,261000,F,3,50.8005,-1.1029,0,0
3,135000,F,5,51.9100,0.9163,0,1
4,370000,S,8,52.4332,-1.5664,0,1
...,...,...,...,...,...,...,...
49995,215000,S,6,53.4695,-2.5806,0,1
49996,90000,S,4,53.2294,-1.3998,0,1
49997,328000,S,9,53.5005,-2.3537,0,1
49998,360000,O,6,54.8338,-1.8117,0,1


The final step is to convert the property type, which contains five categories. Still deploy the same method but list the conversion of all the 5 categorical columns in dummy columns.  

In [23]:
df_cleaned = pd.get_dummies(df_numeric2,
                     columns = ['Property_Type'])

df_cleaned

,Price,Month,Latitude,Longitude,Building_Age,Tenure,Property_Type_D,Property_Type_F,Property_Type_O,Property_Type_S,Property_Type_T
0,72500,4,54.5575,-0.9597,0,1,0,0,0,0,1
1,110000,3,53.0079,-2.1951,0,1,0,0,0,1,0
2,261000,3,50.8005,-1.1029,0,0,0,1,0,0,0
3,135000,5,51.9100,0.9163,0,1,0,1,0,0,0
4,370000,8,52.4332,-1.5664,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
49995,215000,6,53.4695,-2.5806,0,1,0,0,0,1,0
49996,90000,4,53.2294,-1.3998,0,1,0,0,0,1,0
49997,328000,9,53.5005,-2.3537,0,1,0,0,0,1,0
49998,360000,6,54.8338,-1.8117,0,1,0,0,1,0,0


In [25]:
# check out the new dataFrame
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49977 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Price            49977 non-null  int64  
 1   Month            49977 non-null  int64  
 2   Latitude         49977 non-null  float64
 3   Longitude        49977 non-null  float64
 4   Building_Age     49977 non-null  uint8  
 5   Tenure           49977 non-null  uint8  
 6   Property_Type_D  49977 non-null  uint8  
 7   Property_Type_F  49977 non-null  uint8  
 8   Property_Type_O  49977 non-null  uint8  
 9   Property_Type_S  49977 non-null  uint8  
 10  Property_Type_T  49977 non-null  uint8  
dtypes: float64(2), int64(2), uint8(7)
memory usage: 2.2 MB


## 5. Methodology

## 6. Results and Discussion

## 7. Conclusion

In [ ]:
end_time = time.time()
run_time = end_time - start_time
print("Total run time: ", run_time, "seconds")

*(word count: )*

## Bibliography